In [1]:
import numpy as np
from glob import glob
import pydicom
import re
from tqdm import tqdm_notebook

In [ ]:
from multiprocessing.pool import ThreadPool

In [2]:
train, test = np.load('splited.npy', allow_pickle=True)

In [3]:
root_mask_path = "Y:/LIDC-IDRI_final_masks/LIDC-IDRI-{}.npz"

In [4]:
paths = np.concatenate([train, test])

In [5]:
index = 0
for path in tqdm_notebook(paths):
    dcms = [pydicom.dcmread(path) for path in glob(path+'/*.dcm')]
    dcms.sort(key=lambda dcm: float(dcm.ImagePositionPatient[2]))
    vol = np.array([(dcm.pixel_array * float(dcm.RescaleSlope)) + float(dcm.RescaleIntercept) for dcm in dcms])
    
    case = re.search('LIDC-IDRI-(.+?)/', path).group(1)
    
    mask_path = root_mask_path.format(case)
    
    file = np.load(mask_path)
    mask = file['mask']
    file.close()
    
    slices = np.argwhere(mask.sum(axis=(1,2)) != 0).flatten()
    
    subset = "test"
    if(path in train):
        subset = "train"
    
    for slice_ in slices:
        np.savez_compressed("./data/{}/{}.npz".format(subset, index), data=vol[slice_], mask=mask[slice_])
        index += 1

In [6]:
import subprocess

subprocess.call("shutdown -s")

0